In [1]:
'''Example script to generate text from Nietzsche's writings.
At least 20 epochs are required before the generated text
starts sounding coherent.
It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.
If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.
'''

from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io


Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = io.open(path, encoding='utf-8').read().lower()
print('corpus length:', len(text))

corpus length: 600893


In [3]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 57


In [4]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 200285


In [5]:
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


In [6]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

Build model...


In [7]:
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [8]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [9]:
def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [10]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [11]:
model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

Epoch 1/60
200192/200285 [============================>.] - ETA: 0s - loss: 1.9951
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "tation of the superstitious and
magic-de"
tation of the superstitious and
magic-destine and the say and and and in the say the same and the say the say, and and and and in the say to the say and a the spirit and and a most an and and in the say and and the self-experience and the say and and and in the some the serve and the say, and and and the say the some and contion of the say and the some and and and the say to all the say and and and in the waster and in the say in the sa
----- diversity: 0.5
----- Generating with seed: "tation of the superstitious and
magic-de"
tation of the superstitious and
magic-dest the satief of a man firm hid the contionly pradain in the tomines the seever somether and art partine the
the art prisite the parting the seem as the spered which which a great and heart farl and a to the say suc

world mensayiw halgy and a higher and step nof arismons word nature
rowe the than of the abstruce
would will would happinessed
pri: thy man's batill, and losituless this
completed is. the intother seey etistics
-one of the other. he is is ourden taken--with others
for freednessful as amone danger. gnature; is their faiung--took as every se" how to own moricknorded the mitutureruld manay,e, the very
maken-
----- diversity: 1.2
----- Generating with seed: " not sufficient religion in the
world me"
 not sufficient religion in the
world men, to and souls. all a thary saporrate wuck-danger or fhalle of
faculh in all futy in extendamened to estake do,f,
thatre.

.
y-yown, al ob. who harsly extinegeat earphineute permaste prilere fi-esthaace and are ceasion inractably invent feel"n inwainly begrate tom one approfor the their
vivivust,
even are
as the des involushationiar, brequitye-pecratte."

is hidhers,
ecting, in scasptican as, as 
200285/200285 [==============================] - 174s 871u

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:4: RuntimeWarning: divide by zero encountered in log


 is to the degerman moral experiences and creating of its himself and more scientific wanting th
----- diversity: 1.0
----- Generating with seed: "rein--namely, the
immediate succession o"
rein--namely, the
immediate succession of custom and case on the spirit was conditionrous of ascerally the only someby world peoknes and
devation of deptical
duling dimbaraine for
arrapth which see. contimucy of these edipty, the greated with a
huritation for the confisceration has beorm them ever advainte, by relegive
spoksomed, who seem new badly usen" the psychologopace, who has been the ove leason man besieng buitselo of makest, fec
----- diversity: 1.2
----- Generating with seed: "rein--namely, the
immediate succession o"
rein--namely, the
immediate succession of effory, in that
metaphylars onc! inwindt should wons this suptre, severurely one consciousn, thet
humanity, in osalves of
resulafs   sudy tooknest
for, and world, expectience of an i humanceble origious called inasexts them about
the si

Epoch 10/60
200192/200285 [============================>.] - ETA: 0s - loss: 1.3989
----- Generating text after Epoch: 9
----- diversity: 0.2
----- Generating with seed: "ceives, perhaps, that those effects coul"
ceives, perhaps, that those effects could not a results of the something of the consideral and self-said, and a still and still of the sensible of the seems to the problem of the morality and the consist of the consideral and that is a surched the seems to the sensible of the problem of the sensible of the the problem of the self-seems to the their problem of the seems to the superiority of the sensible of the results of the problem of 
----- diversity: 0.5
----- Generating with seed: "ceives, perhaps, that those effects coul"
ceives, perhaps, that those effects could no owe will that is the ligh to some fitle of the and desereble sense, and a sort of the resting in the encist among every of the feelings and such a surpenting them have action to the consideration of the saint,

collechient of ndedibility as a places chur to all tutherwaks a god general certain reperposest doforment for the former out of something doing to one should that remain reflectuous
infinementialer, for individual on its races so, who
it of evilficrly of the finally opposing subwlermic assultal oovity regart-dusspendile occhimest culturenally in a fact, religion pain?
"the net former when devoluntable
----- diversity: 1.2
----- Generating with seed: "he shades of these troubled spirits
coll"
he shades of these troubled spirits
collecrihe, and must precisely thamen merely means.


61e by the soul moeen--     yee,
the surmbilstrus which a
presen us slead romanify only, is suming roke
and der letks in the ""man." or when thereaon-constic, and "doorhpoced," could adking the honcular protic
or dehorm of worse is broudmer than the shareness
frient suffice instinnt than this gyismhis
have all music then they curpaining than it mus
200285/200285 [==============================] - 121s 605us/st

without thinking very much about it, so a more the sensible of the sense, that is the same that it is all the senses of the sense of the self interpreteness of the sensible of the sensible in the senses of the same that in the sense of the sensible of the sense, and the seem to the sense, and self-anguined in the same that is the sense of the far and the sensible of the sense, and all that is not be a strength of the sense, and and in
----- diversity: 0.5
----- Generating with seed: "y
without thinking very much about it, s"
y
without thinking very much about it, such a man in the scientific sensible interpreted that the same martion that it is all the rese to the greater that in the agree of the superecience that is it is not the same time there is a desire of the sensations of its extanted of the man is man of the fully of the ideal, in the father it is some intellectual and secreted to the artists. it is the really concealed the contradicting with it is 
----- diversity: 1.0
----- G

KeyboardInterrupt: 